In [3]:
hold_period = 1 # hold period in days
threshold = -70 # price difference threshold in percent maybe try -30

#how prices (or dollar-volumes) jump between close one trading day to the open of the next. if the dollar-volume
#drops by more than a set percentage between close and open, buy then sell it the next day, at opening. 

def initialize(context):
    
    context.stocks = [sid(8554)] # SPY

    context.previous_price = 1

    context.initialize = True
    context.event_day = 0
    context.new_day = False
    context.day_counter = 0
    context.bought = False
    context.day_submitted = 0
    context.num_shares = 0
    set_commission(commission.PerTrade(cost=1.0))
  
def handle_data(context, data):

    current_price = data[context.stocks[0]].price
    num_shares = context.portfolio.cash/current_price

    current_price = current_price*data[context.stocks[0]].volume

    event_day = data[context.stocks[0]].datetime.day

    if context.initialize:
        context.event_day = event_day
        context.initialize = False 

    if event_day != context.event_day:
        context.new_day = True
    else:
        context.new_day = False

    if context.new_day:
        context.day_counter = context.day_counter + 1
        price_diff = 100*(current_price/context.previous_price-1)

        if price_diff < 0:
            record(price_diff = price_diff)

        if (price_diff < threshold) and not(context.bought):
            order(context.stocks[0],num_shares)
            context.bought = True
            context.num_shares = num_shares
            print ('Bought '+str(num_shares)+' shares')
            context.day_submitted = context.day_counter
        elif context.day_counter-context.day_submitted == hold_period and context.bought:
            order(context.stocks[0],-context.num_shares)
            context.bought = False
            print ('Sold '+str(context.num_shares)+' shares')

    context.previous_price = current_price
    context.event_day = event_day